In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

def get_sampled_file_paths(directory, sample_fraction=0.002):
    all_classes = os.listdir(directory)
    sampled_file_paths = []
    labels = []

    for class_name in all_classes:
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            all_files = os.listdir(class_path)
            num_files_to_select = int(len(all_files) * sample_fraction)
            sampled_files = np.random.choice(all_files, size=num_files_to_select, replace=False)
            for file_name in sampled_files:
                sampled_file_paths.append(os.path.join(class_name, file_name))
                labels.append(class_name)

    return sampled_file_paths, labels

train_sampled_file_paths, train_labels = get_sampled_file_paths(train_dir)

test_sampled_file_paths, test_labels = get_sampled_file_paths(test_dir)

df_train = pd.DataFrame({
    'filename': [os.path.join(train_dir, path) for path in train_sampled_file_paths],
    'class': train_labels
})

df_test = pd.DataFrame({
    'filename': [os.path.join(test_dir, path) for path in test_sampled_file_paths],
    'class': test_labels
})

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

validation_generator = validation_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    y_col='class',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

print(train_generator.samples)
print(validation_generator.samples)

Found 100 validated image filenames belonging to 10 classes.
Found 20 validated image filenames belonging to 10 classes.
100
20


In [2]:
from tensorflow.keras.applications import VGG16

# Użycie pretrenowanego modelu VGG16 bez górnych warstw
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Zamrażamy wagi pretrenowanych warstw
for layer in base_model.layers:
    layer.trainable = False

# Dodanie własnych warstw |na wierzchu VGG16
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),  # Regularizacja L2
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Kompilacja modelu
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)

Epoch 1/50


/Users/dawid/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 43s 12s/step - accuracy: 0.2063 - loss: 15.4969 - val_accuracy: 0.1000 - val_loss: 17.7159
Epoch 2/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 29s 15s/step - accuracy: 0.0625 - loss: 18.2155

2024-08-19 20:08:20.307851: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/dawid/.pyenv/versions/3.11.8/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


3/3 ━━━━━━━━━━━━━━━━━━━━ 30s 8s/step - accuracy: 0.0625 - loss: 18.2155 - val_accuracy: 0.1000 - val_loss: 14.2164
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 52s 15s/step - accuracy: 0.1179 - loss: 16.9557 - val_accuracy: 0.1000 - val_loss: 9.9945
Epoch 4/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 32s 16s/step - accuracy: 0.0938 - loss: 13.3674

2024-08-19 20:09:44.641494: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


3/3 ━━━━━━━━━━━━━━━━━━━━ 27s 6s/step - accuracy: 0.0938 - loss: 13.3674 - val_accuracy: 0.1000 - val_loss: 8.6746
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 65s 24s/step - accuracy: 0.1419 - loss: 12.3540 - val_accuracy: 0.1500 - val_loss: 8.8203
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 5s/step - accuracy: 0.0000e+00 - loss: 13.2051 - val_accuracy: 0.1000 - val_loss: 9.1084
Epoch 7/50
1/3 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.0000e+00 - loss: 9.4736

In [10]:
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // 32)
print(f'Test accuracy: {test_acc:.2f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 649ms/step - accuracy: 0.1000 - loss: 2.3034
Test accuracy: 0.10


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(150, 150), color_mode='rgba')  # Obsługa plików PNG
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    class_labels = list(train_generator.class_indices.keys())
    return class_labels[predicted_class]

img_path = 'data/train/sztuka nowoczesna/Screenshot 2024-07-25 at 01.22.33.png'
predicted_class = predict_image(img_path, model)
print(f'Ten obrazek przedstawia styl: {predicted_class}')

In [ ]:
from ipywidgets import widgets
from IPython.display import display
import PIL.Image

def on_upload_change(change):
    img = PIL.Image.open(change['new'][0])
    img_path = 'uploaded_image.png'
    img.save(img_path)
    display(img)
    predicted_class = predict_image(img_path, model)
    print(f'Ten obrazek przedstawia styl: {predicted_class}')

upload_widget = widgets.FileUpload(accept='image/*', multiple=False)
upload_widget.observe(on_upload_change, names='value')
display(upload_widget)